# Yemen Al Hodeydah Accessibility villages to education with Adv snapping

In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.ops import split, unary_union
from shapely.geometry import box, Point

import matplotlib.pyplot as plt

In [68]:
# Get reference to GOSTNets
sys.path.append(r'../../../../GOSTnets')
import GOSTnets as gn

In [69]:
from GOSTnets.load_osm import *

## Import clipped Graphs w/ speeds!

In [70]:
# regular graph
G2_time = nx.read_gpickle(r'input/saved_graphs/yemen_regular_graph_w_speeds_hodeydah_clipped.pickle')

In [71]:
print(nx.info(G2_time))

Name: 
Type: MultiDiGraph
Number of nodes: 12175
Number of edges: 30995
Average in degree:   2.5458
Average out degree:   2.5458


In [72]:
# rehab graph
G2_time_rehab = nx.read_gpickle(r'input/saved_graphs/yemen_rehab_graph_w_speeds_hodeydah_clipped.pickle')

In [73]:
print(nx.info(G2_time_rehab))

Name: 
Type: MultiDiGraph
Number of nodes: 12175
Number of edges: 30995
Average in degree:   2.5458
Average out degree:   2.5458


In [74]:
gn.example_edge(G2_time,3)

(0, 1, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000019A23405430>, 'id': 99289, 'one_way': False, 'osm_id': '463039122', 'Type': 'legitimate', 'infra_type': 'residential', 'key': 'edge_99289', 'length': 80.134174811368, 'name': None, 'time': 14.42415146604624, 'mode': 'drive'})
(0, 2, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000019A234054C0>, 'id': 99290, 'one_way': False, 'osm_id': '463039123', 'Type': 'legitimate', 'infra_type': 'residential', 'key': 'edge_99290', 'length': 28.314800532222, 'name': None, 'time': 5.09666409579996, 'mode': 'drive'})
(0, 9365, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000019A234054F0>, 'id': 99288, 'one_way': False, 'osm_id': '463039122', 'Type': 'legitimate', 'infra_type': 'residential', 'key': 'edge_99288', 'length': 144.782765304676, 'name': None, 'time': 26.060897754841676, 'mode': 'drive'})


## Import Origins

In [75]:
#origins = gpd.read_file(r'input/yemen_population_worldpop2020.shp')
#origins = gpd.read_file(r'input/hotosm_yem_populated_places_points.shp')
origins = gpd.read_file(r'input/hodeidah_datasets/hotosm_yem_populated_places_points_hodeidah.shp')

In [76]:
origins['osm_id'] = origins['osm_id'].astype('int64')

In [77]:
origins

,osm_id,population,place,name,source,is_in,geometry
0,1217802271,None,hamlet,موشج,None,None,POINT (43.28081 13.71816)
1,1217802273,1313,village,الكديف,None,None,POINT (43.28551 13.70957)
2,1217802278,None,isolated_dwelling,العاقر,None,None,POINT (43.29094 13.72292)
3,1217802295,1392,village,الوعرة,None,None,POINT (43.26202 13.76943)
4,1217805405,103,hamlet,العكــش,None,None,POINT (43.46556 13.95102)
...,...,...,...,...,...,...,...
4158,7294970363,None,hamlet,محلّ عقيل,None,None,POINT (43.23014 15.31383)
4159,7294970365,None,hamlet,محلّ دهنة,None,None,POINT (43.14946 15.62061)
4160,7294970369,None,hamlet,المقشم محلّ الدوم,None,None,POINT (43.13768 15.18068)
4161,7294970372,166,hamlet,جميشة,None,None,POINT (43.07813 14.81021)


In [78]:
# find graph utm zone
G_utm = gn.utm_of_graph(G2_time)
G_utm

'+proj=utm +zone=38 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'

## advanced snapping
Threshold is 5km

In [79]:
G_adv, pois_meter, new_footway_edges = gn.advanced_snap(G2_time, origins, u_tag = 'stnode', v_tag = 'endnode', node_key_col='osmid', edge_key_col='osm_id', poi_key_col='osm_id', road_col = 'infra_type', oneway_tag = 'one_way', path='output/output_edu', threshold=5000, knn=5, measure_crs=G_utm, factor = 1, verbose = True)

Building rtree...
finished Building rtree
seconds elapsed: 2.0754995346069336
updating external nodes (pois)
finished updating external nodes (pois)
seconds elapsed: 2.0805015563964844
Projecting POIs to the network...2
finished pois_meter['near_idx'] and pois_meter['near_lines']
seconds elapsed: 3.657855987548828
finished pois_meter['kne_idx']
seconds elapsed: 3.830895185470581
assigning a projected point to each POI
finished assigning a projected point to each POI
seconds elapsed: 3.941920042037964
Updating internal nodes...
finished Updating internal nodes
seconds elapsed: 4.447034120559692
print _new_nodes
Updating internal edges...
finished creating line_pps_dict
seconds elapsed: 4.652080535888672
creating new_lines
finished creating new_lines
seconds elapsed: 5.130188226699829
Updating update_edges
debugging
finished Updating update_edges
seconds elapsed: 5.788341999053955
Updating external links...
debugging
node count before: 19460
node count after: 19231
pois_meter count befor

In [80]:
originNodes = list(pois_meter['osm_id'])

### Add walking time to new footway edges

In [81]:
yemen_travel_times = {
'motorway': 40,
'motorway_link': 40,
'trunk': 40,
'trunk_link':40,
'primary': 37.5,
'primary_link': 37.5,
'secondary': 35,
'secondary_link': 35,
'tertiary':30,
'tertiary_link':30,
'residential': 20,
'unclassified': 20,
'projected_footway':3.5
}

## Add walking time to new footway edges
#### convert_network_to_time has to also be re-run because advanced snapping altered the edges

In [82]:
# convert_network_to_time function now has 'projected_footway' as a highway class
G_adv_w_time = gn.convert_network_to_time(G_adv, 'length', road_col = 'infra_type', speed_dict = yemen_travel_times)

C:\repos\GOST_PublicGoods\Implementations\FY22\ACC_yemen_accessibility\../../../../GOSTnets\GOSTnets\core.py:1016: UserWarning: Aree you sure you want to convert length to time? This graph already has a time attribute
  warnings.warn('Aree you sure you want to convert length to time? This graph already has a time attribute')


In [83]:
gn.example_edge(G_adv_w_time, 3)

(0, 1, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000019A3AF653D0>, 'one_way': False, 'osm_id': '463039122', 'mode': 'drive', 'id': 99289.0, 'name': None, 'time': 14.42415146604624, 'infra_type': 'residential', 'key': 'edge_99289', 'Type': 'legitimate', 'length': 80.134174811368})
(0, 2, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000019A3AF65A30>, 'one_way': False, 'osm_id': '463039123', 'mode': 'drive', 'id': 99290.0, 'name': None, 'time': 5.09666409579996, 'infra_type': 'residential', 'key': 'edge_99290', 'Type': 'legitimate', 'length': 28.314800532222})
(0, 9365, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000019A3C43A040>, 'one_way': False, 'osm_id': '463039122', 'mode': 'drive', 'id': 99288.0, 'name': None, 'time': 26.060897754841676, 'infra_type': 'residential', 'key': 'edge_99288', 'Type': 'legitimate', 'length': 144.782765304676})


## Advanced snapping to rehab graph

In [84]:
G_adv_rehab, pois_meter_rehab, new_footway_edges_rehab = gn.advanced_snap(G2_time_rehab, origins, u_tag = 'stnode', v_tag = 'endnode', node_key_col='osmid', poi_key_col='osm_id', edge_key_col='osm_id', road_col = 'infra_type', path='output', threshold=5000, knn=5, measure_crs=G_utm, factor = 1, verbose = True)

Building rtree...
finished Building rtree
seconds elapsed: 2.034458637237549
updating external nodes (pois)
finished updating external nodes (pois)
seconds elapsed: 2.0384600162506104
Projecting POIs to the network...2
finished pois_meter['near_idx'] and pois_meter['near_lines']
seconds elapsed: 3.584808349609375
finished pois_meter['kne_idx']
seconds elapsed: 3.7518458366394043
assigning a projected point to each POI
finished assigning a projected point to each POI
seconds elapsed: 3.862870931625366
Updating internal nodes...
finished Updating internal nodes
seconds elapsed: 4.367985725402832
print _new_nodes
Updating internal edges...
finished creating line_pps_dict
seconds elapsed: 4.577032566070557
creating new_lines
finished creating new_lines
seconds elapsed: 5.045138359069824
Updating update_edges
debugging
finished Updating update_edges
seconds elapsed: 5.700286149978638
Updating external links...
debugging
node count before: 19460
node count after: 19231
pois_meter count befor

### Add walking time to new footway edges
convert_network_to_time has to also be re-run because advanced snapping altered the edges

In [85]:
# convert_network_to_time function now has 'projected_footway' as a highway class
G_adv_w_time_rehab = gn.convert_network_to_time(G_adv_rehab, 'length', road_col = 'infra_type', speed_dict = yemen_travel_times)

C:\repos\GOST_PublicGoods\Implementations\FY22\ACC_yemen_accessibility\../../../../GOSTnets\GOSTnets\core.py:1016: UserWarning: Aree you sure you want to convert length to time? This graph already has a time attribute
  warnings.warn('Aree you sure you want to convert length to time? This graph already has a time attribute')


### Alter times on rehab roads for rehab graph

In [86]:
count=0
for u, v, data in G_adv_w_time_rehab.edges(data=True):
    if 'Type' in data:
        if data['Type'] == 'rehab':
            print(data['Type'])
            count+=1
            # data['length'] is in meters
            # speed after rehab is 80 km per hr
            kmph = (data['length'] / 1000) / 80
            in_seconds = kmph * 60 * 60
            print(f"time before is {data['time']}")
            data['time'] = in_seconds
            print(f"time after is {data['time']}")

rehab
time before is 214.63516334767507
time after is 53.65879083691877
rehab
time before is 172.70061777433872
time after is 43.17515444358468
rehab
time before is 172.70061777433872
time after is 43.17515444358468
rehab
time before is 456.60439936836866
time after is 114.15109984209217
rehab
time before is 186.5377814900537
time after is 46.634445372513426
rehab
time before is 111.14317354174379
time after is 27.785793385435948
rehab
time before is 58.84300455216477
time after is 14.710751138041193
rehab
time before is 111.14317354174379
time after is 27.785793385435948
rehab
time before is 74.87782624966796
time after is 18.71945656241699
rehab
time before is 260.203229918301
time after is 65.05080747957525
rehab
time before is 48.919778635050825
time after is 12.229944658762706
rehab
time before is 7.077693443906293
time after is 1.7694233609765733
rehab
time before is 260.203229918301
time after is 65.05080747957525
rehab
time before is 530.3319008218932
time after is 132.58297520

# Import Destinations

In [87]:
#destinations = gpd.read_file(r'yemen_accessibility_export_package/al_hadaydah_district_pts.shp')
destinations = gpd.read_file(r'input/hodeidah_datasets/hotosm_yem_educational_facilities_points_hodeidah.shp')
#destinations = gpd.read_file(r'input/hodeidah_datasets/HeRAMS_2018_April_snapped_renamed_hodeidah.shp')
#destinations = gpd.read_file(r'input/HeRAMS_2018_April_snapped_renamed.shp')
#destinations = gpd.read_file(r'input/yemen_data_clusters_inland3.shp')

### using regular snapping for destinations using the original network

In [88]:
snapped_destinations_df = gn.pandana_snap_c(G2_time, destinations, source_crs='epsg:4326', target_crs= 'epsg:32638')

### keep the snapped destinations that are less than 10,000 meters from the nearest road node

In [89]:
snapped_destinations_df = snapped_destinations_df[snapped_destinations_df['NN_dist'] < 10000]

In [90]:
snapped_destinations_df[:3]

,osm_id,addrcity,name,amenity,operatorty,addrfull,source,building,capacitype,geometry,NN,NN_dist
0,4.237986e+09,None,مدرسة الصباح الثانوية,school,None,None,None,None,None,POINT (42.95375 14.78448),7859,42.559168
1,4.238026e+09,None,مدرسة الشهيد الخادم غالب الاساسية,school,None,None,None,None,None,POINT (42.95487 14.78639),6917,69.184029
2,4.547319e+09,None,مدرسة الثورة,school,None,None,None,None,None,POINT (42.94512 14.80116),9966,71.143976


In [91]:
#snapped_destinations_merged

In [92]:
destinationNodes = list(snapped_destinations_df['NN'].unique())

## Calculate OD matrix for regular graph

In [93]:
%%time
OD_matrix = gn.calculate_OD(G_adv_w_time, originNodes, destinationNodes, fail_value=-1, weight='time', one_way_roads_exist = True)

Wall time: 4min 3s


In [94]:
closest_facility_per_origin = OD_matrix.min(axis=1)
results = pd.DataFrame([originNodes, closest_facility_per_origin]).transpose()
colName = "travel_time_to_closest_facility"
results.columns = ['osm_id', colName]
results

,osm_id,travel_time_to_closest_facility
0,1.217802e+09,173.762366
1,1.217802e+09,495.425905
2,1.217802e+09,897.871800
3,1.217802e+09,981.914306
4,1.217805e+09,3452.916188
...,...,...
3929,7.294970e+09,1334.216804
3930,7.294970e+09,6977.327863
3931,7.294970e+09,5264.799087
3932,7.294970e+09,2201.916223


In [95]:
# test
#results[results['travel_time_to_closest_facility']==-1]

In [96]:
pois_meter[:3]

,osm_id,population,place,name,source,is_in,geometry,near_idx,near_lines,kne_idx,pp
0,1217802271,None,hamlet,موشج,None,None,POINT (43.28081 13.71816),"[7759, 14378, 7760, 7758, 16967]","7759 LINESTRING (314274.319 1517830.982, 3...",7759,POINT (314018.4500703381 1517245.907939139)
1,1217802273,1313,village,الكديف,None,None,POINT (43.28551 13.70957),"[7760, 2032, 2034, 16967, 7759]","7760 LINESTRING (314898.472 1512903.033, 3...",2032,POINT (314986.5183242566 1516313.591813721)
2,1217802278,None,isolated_dwelling,العاقر,None,None,POINT (43.29094 13.72292),"[13809, 14378, 16967, 7760, 7759]","13809 LINESTRING (310944.634 1525903.122, 3...",14378,POINT (314397.8876868006 1517439.63425154)


In [97]:
output2 = pd.merge(pois_meter, results, on="osm_id")

In [98]:
output2[["travel_time_to_closest_facility"]] = output2[["travel_time_to_closest_facility"]].apply(pd.to_numeric)

In [99]:
output2 = output2.drop(columns=['near_idx','near_lines','pp'], axis=1)

In [100]:
output2['travel_time_to_closest_facility'].mean()

4547.641067237624

In [101]:
destinations_gpd = gpd.GeoDataFrame(output2, crs = "epsg:4326", geometry = 'geometry')
#destinations_gpd.to_file('yemen_accessibility_health_hodeydah.shp')
#destinations_gpd.to_file('yemen_accessibility_education_hodeydah.shp')
#destinations_gpd.to_file('yemen_accessibility_district_centers_wo_walking_time2.shp')
#destinations_gpd.to_file('yemen_accessibility_district_centers_w_walking_time3.shp')
destinations_gpd.to_file('yemen_accessibility_education_hodeydah_w_walking_time3.shp')

C:\Users\valhalla\AppData\Local\Temp/ipykernel_39496/1830115610.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  destinations_gpd.to_file('yemen_accessibility_education_hodeydah_w_walking_time3.shp')


## Calculate OD matrix for graph with rehabilitated roads

In [102]:
%%time
OD_matrix2 = gn.calculate_OD(G_adv_w_time_rehab, originNodes, destinationNodes, fail_value=-1, weight='time', one_way_roads_exist = True)
closest_facility_per_origin2 = OD_matrix2.min(axis=1)
results2 = pd.DataFrame([originNodes, closest_facility_per_origin2]).transpose()
colName = "travel_time_to_closest_facility"
results2.columns = ['osm_id', colName]

Wall time: 4min 5s


In [103]:
output3 = pd.merge(pois_meter, results2, on="osm_id")
output3[["travel_time_to_closest_facility"]] = output3[["travel_time_to_closest_facility"]].apply(pd.to_numeric)

In [104]:
output3[:3]

,osm_id,population,place,name,source,is_in,geometry,near_idx,near_lines,kne_idx,pp,travel_time_to_closest_facility
0,1217802271,None,hamlet,موشج,None,None,POINT (43.28081 13.71816),"[7759, 14378, 7760, 7758, 16967]","7759 LINESTRING (314274.319 1517830.982, 3...",7759,POINT (314018.4500703381 1517245.907939139),173.762366
1,1217802273,1313,village,الكديف,None,None,POINT (43.28551 13.70957),"[7760, 2032, 2034, 16967, 7759]","7760 LINESTRING (314898.472 1512903.033, 3...",2032,POINT (314986.5183242566 1516313.591813721),495.425905
2,1217802278,None,isolated_dwelling,العاقر,None,None,POINT (43.29094 13.72292),"[13809, 14378, 16967, 7760, 7759]","13809 LINESTRING (310944.634 1525903.122, 3...",14378,POINT (314397.8876868006 1517439.63425154),897.871800


In [105]:
output3['travel_time_to_closest_facility'].mean()

4510.639170188976

In [106]:
output3 = output3.drop(columns=['near_idx','near_lines','pp'], axis=1)

In [107]:
destinations_gpd = gpd.GeoDataFrame(output3, crs = "epsg:4326", geometry = 'geometry')
#destinations_gpd.to_file('yemen_accessibility_health_hodeydah_rehabbed.shp')
#destinations_gpd.to_file('yemen_accessibility_education_hodeydah_rehabbed.shp')
#destinations_gpd.to_file('yemen_accessibility_district_centers_rehabbed_wo_walking_time2.shp')
#destinations_gpd.to_file('yemen_accessibility_district_centers_rehabbed_w_walking_time3.shp')
destinations_gpd.to_file('yemen_accessibility_education_rehabbed_hodeydah_w_walking_time3.shp')

C:\Users\valhalla\AppData\Local\Temp/ipykernel_39496/1272663126.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  destinations_gpd.to_file('yemen_accessibility_education__rehabbed_hodeydah_w_walking_time3.shp')


## Calculate average travel time per Rehabilitated road

In [108]:
OD_df_reg = pd.DataFrame(OD_matrix, index = originNodes, columns = destinationNodes)

In [109]:
closest_facility_id_per_origin = OD_df_reg.idxmin(axis=1)

In [110]:
closest_facility_id_per_origin

1217802271     3935
1217802273    11446
1217802278    11446
1217802295     3202
1217805405      554
              ...  
7294970362       29
7294970363       29
7294970365       36
7294970372      684
7784667539      963
Length: 3934, dtype: int64

In [111]:
OD_matrix2.shape

(3934, 50)

In [112]:
OD_df_rehab = pd.DataFrame(OD_matrix2, index = originNodes, columns = destinationNodes)

In [113]:
closest_facility_id_per_origin_rehab = OD_df_rehab.idxmin(axis=1)

In [114]:
from collections import defaultdict

In [115]:
def group(lst, n):
  for i in range(0, len(lst), 1):
    val = lst[i:i+n]
    if len(val) == n:
      yield tuple(val)

In [116]:
#https://stackoverflow.com/questions/26367812/appending-to-list-in-python-dictionary
rehab_road_tracker = defaultdict(list)

In [117]:
rehab_road_tracker

defaultdict(list, {})

In [118]:
# save the rehab_origin_destination_pairs to compare the same edges using the regular graph later
rehab_origin_destination_pairs = []

In [119]:
def look_up(shortest_path, shortest_path_time, row, col):
    edge_segements = list(group(shortest_path,2))
    for edge in edge_segements:
        try:    
            if G_adv_w_time_rehab[edge[0]][edge[1]][0]['Type'] == 'rehab':
                print('rehab match')
                road_name = G_adv_w_time_rehab[edge[0]][edge[1]][0]['name']
                rehab_road_tracker[road_name].append(shortest_path_time)
                rehab_origin_destination_pairs.append((row, col))
                break
        except:
            print(G_adv_w_time_rehab[edge[0]][edge[1]][0])
            print(edge[0])
            print(edge[1])
            global mismatch_count
            mismatch_count += 1

In [120]:
closest_facility_id_per_origin_rehab

1217802271     3935
1217802273    11446
1217802278    11446
1217802295     3202
1217805405      554
              ...  
7294970362       29
7294970363       29
7294970365       36
7294970372      684
7784667539      963
Length: 3934, dtype: int64

In [121]:
%time
mismatch_count = 0
for row, col in closest_facility_id_per_origin_rehab.items():
    print(row)
    print(col)
    shortest_path = nx.dijkstra_path(G_adv_w_time_rehab, row, col, weight='time')
    shortest_path_time = OD_df_rehab.loc[row, col]
    look_up(shortest_path, shortest_path_time, row, col)

Wall time: 0 ns
1217802271
3935
1217802273
11446
1217802278
11446
1217802295
3202
1217805405
554
1217805410
554
1217805420
554
1217805421
554
1217805430
554
1217805434
554
1217805439
554
1217806499
554
1217806501
554
1217806509
554
1217806521
554
1217806522
554
1217806528
554
1217806537
554
1217806538
554
1217806544
554
1217806549
554
1217806556
554
1217806557
554
1217806567
554
1217806576
554
1217806586
554
1217806588
554
1217812359
554
1217812378
554
1217812384
554
1217812887
554
1217812889
554
1217812899
554
1217812900
554
1217812902
554
1217812911
554
1217812917
554
1217812918
554
1217812922
554
1217814196
554
1217814214
554
1217814216
554
1217814233
554
1217814243
554
1217814244
554
1217814262
554
1217814264
554
1217814285
554
1217814298
554
1217814309
554
1217814314
554
1217814332
554
1217814345
554
1217814363
554
1217814381
554
1217814391
554
1217814398
554
1217814407
554
1217814432
554
1217814446
554
1217814452
554
1217814473
554
1217814500
554
1217814520
554
1217851343
554
121

1221857203
8525
1221857244
6152
1221857331
6152
1221857356
554
1221857382
6152
1221857393
6152
1221857428
6152
1221857439
8525
1221857472
6152
1221857540
6152
1221857556
6152
1221863814
6152
1221863855
6152
1221863892
8525
1221863895
8525
1221863933
6152
1221863938
6152
1221863991
8525
1221864038
6152
1221864043
8525
1221864083
6152
1221864089
6152
1221864112
8525
1221864119
6152
1221864156
8525
1221864165
6152
1221864170
6152
1221864214
6152
1221864264
6152
1221864274
8525
1221864309
6152
1221864324
6152
1221864362
8525
1221864376
6152
1221864409
8525
1221877138
963
1221877244
963
1221877264
963
1221877354
963
1221877398
963
1221877425
963
1221877449
963
1221877501
963
1221877533
963
1221877584
963
1221877610
963
1221877748
963
1221877813
963
1221877827
963
1221877893
963
1221877955
963
1221878019
963
1221878032
963
1221878092
963
1221878137
963
1221878189
963
1221878228
963
1221878307
963
1221880619
963
1221880692
963
1221880705
963
1221880748
963
1221880795
963
1221880823
963
122188

1224156847
29
1224156857
29
1224156859
29
1224160523
29
1224160537
29
1224160545
29
1224160550
29
1224160555
29
1224160563
29
1224160573
29
1224160581
29
1224160584
29
1224160594
29
1224160599
29
1224160607
29
1224160628
29
1224160636
29
1224160639
29
1224160655
29
1224160660
29
1224160665
29
1224160674
29
1224160676
29
rehab match
1224160688
29
1224160697
29
1224160698
29
1224160883
29
1224160894
29
rehab match
1224160895
29
1224160896
29
1224160919
29
1224160936
29
rehab match
1224160938
29
1224160949
29
1224160950
29
1224160955
29
1224160956
29
1224160966
29
1224162207
29
1224162209
29
1224162212
29
1224162224
29
1224162225
29
1224162234
29
1224162250
29
1224162254
29
1224162265
29
1224162267
29
1224162268
29
1224162275
29
1224162282
29
1224162284
29
1224174108
29
1224174149
29
1224174154
36
1224177160
36
1224177173
36
1224177188
36
1224177205
36
1224177778
36
1224177781
36
1224177786
36
1224177796
36
1224177800
36
1224177802
36
1224177803
36
1224177811
1419
1224177814
29
1224177815

6936501276
29
6936501277
29
6936501278
29
6936501282
29
6936501283
29
6936501284
29
6936501285
29
6936501286
29
6936501287
29
6936501288
29
6936501289
29
6936501290
29
6936501468
4482
6936501691
29
6936503068
29
6936503674
29
6936503675
4482
6936503806
29
6936504593
4482
6936504594
4482
6936504595
4482
6936504596
29
6936504597
963
rehab match
6936504598
963
rehab match
6936504599
684
6936504600
963
rehab match
6936504602
4482
6936504609
4482
6936504610
4482
6936504611
4482
6936504615
4482
6936504616
4482
6936504617
4482
6936504618
29
6936504619
29
6936504626
29
6936504627
29
6936504628
29
6936504629
4482
6936504630
29
6936504632
4482
6936504633
29
6936504634
29
6936504635
29
6936504636
29
6936504637
29
6936504638
29
6936504645
684
6936504647
684
6936504652
963
6936504653
4482
6936504654
4482
6936504655
684
6936504656
4482
6936504658
29
6936504659
29
6936504660
29
6936504662
29
6936504663
29
6936504664
29
6936504665
29
6936504666
29
6936504667
29
6936504668
29
6936504669
29
6936504671
2

6936522763
10930
6936522793
554
6936523113
9202
6936523114
554
6936523115
554
6936523116
554
6936523117
9088
6936523180
29
6936523184
684
6936523226
29
6936523242
29
6936523247
29
6936523248
29
6936523258
29
6936523259
29
6936523276
29
6936523277
29
6936523315
554
6936523316
554
6936523331
1419
6936523337
1419
6936523349
554
6936523367
554
6936523398
963
6936523414
6152
6936523417
6152
6936523509
6152
6936523510
6152
6936523512
6152
6936523513
963
6936523514
963
6936523680
963
6936523738
963
6936523739
684
6936523740
684
6936523741
29
6936523742
29
6936523743
4482
6936523744
4482
6936523745
29
6936523746
684
6936523747
684
6936523748
4482
6936523749
29
6936523750
29
6936523751
29
6936523752
29
6936523753
29
6936523754
684
6936523756
684
6936523757
29
6936523758
29
6936523759
29
6936523760
29
6936523761
684
6936523762
4482
6936523763
4482
6936523764
29
6936523765
29
6936523766
4482
6936523767
4482
6936523768
4482
6936523769
4482
6936523770
684
6936523771
684
6936523772
4482
6936523773
2

6936539371
1419
6936539372
1419
6936539373
1419
6936539374
1419
6936539375
1419
6936539681
29
6936539685
29
6936539697
1419
6936542161
684
6936542937
554
6936542938
554
6936542940
36
6936542941
36
6936542945
36
6936542967
36
6936542969
10930
6936542970
36
6936543031
554
6936543054
684
6936543055
29
6936543056
29
6936543058
29
6936543059
29
6936543061
684
rehab match
6936543062
684
rehab match
6936543063
4482
6936543064
29
6936543065
29
6936543066
4482
6936543067
4482
6936543069
684
6936543070
684
6936543072
963
6936543073
29
6936543074
684
6936543075
4482
6936543077
29
6936543079
29
6936543080
36
6936543081
4482
6936543082
4482
6936543083
29
6936543084
4482
6936543085
29
6936543089
29
6936543090
29
6936543091
7076
6936543093
29
6936543180
36
6936543213
36
6936543214
36
6936543215
36
6936543216
36
6936543220
36
6936543223
10930
6936543224
10930
6936543261
29
6936543266
29
6936543267
29
6936543268
29
6936543269
29
6936543270
29
6936543348
29
6936543486
10930
6936543524
10930
6936543563
3

7294949789
963
7294949793
29
7294949794
963
7294949798
29
7294949800
963
7294949801
963
7294949802
29
rehab match
7294949803
36
7294949804
4482
7294949805
684
7294949806
963
7294949807
4482
7294949808
684
7294949809
29
7294949811
29
7294949822
36
7294949827
963
7294949829
4482
7294949830
4482
7294949831
963
7294949832
29
7294949834
4482
7294949837
29
7294949841
7076
7294949843
29
rehab match
7294949846
29
7294949847
963
7294949849
29
7294949850
4482
7294949852
36
7294949858
1419
7294949865
684
7294949869
684
7294949870
963
7294949871
29
rehab match
7294949874
29
7294949879
29
7294949890
29
7294949891
29
7294949892
963
7294949898
963
7294949899
963
7294949910
29
7294949923
4482
7294949926
4482
7294949928
1419
7294949933
29
7294949953
8525
7294949992
963
rehab match
7294949996
29
7294949998
9088
7294950002
963
7294950007
36
7294950009
29
7294950055
4482
7294950059
4482
7294950061
36
7294950063
29
rehab match
7294950065
29
7294950067
29
7294950071
8525
7294950072
29
7294950076
963
7294950

In [122]:
# they are the edges that got clipped at the boundaries
mismatch_count

0

In [123]:
rehab_road_tracker

defaultdict(list,
            {'mansuriah_dir_maalm': [3979.5536776970957,
              4663.80148852058,
              4869.465751760317,
              3578.7549589788396,
              4318.526688838023,
              5771.398079635896,
              5074.928041406306,
              5005.585209870586,
              5374.017532204648,
              5271.381900804423,
              4453.9368414916025,
              3551.6823865133592,
              3636.6294866993003,
              5049.124669064895,
              3477.2609302155615,
              5077.673055709295,
              5075.83357900716,
              4226.984674323532,
              5276.636459436672,
              5097.192468734656,
              6095.968028950706,
              3568.096147323682,
              3872.060592205872,
              4839.133753151417,
              6981.639756612992,
              6930.503112308578,
              6814.653763064684,
              4078.6752424163296,
              6404.00404452505

In [124]:
rehab_road_tracker_mean = {}

In [125]:
from statistics import mean

In [126]:
for key, value in rehab_road_tracker.items():
    print(key)
    print(mean(value))
    print(len(value))

mansuriah_dir_maalm
5319.328476849615
121
bajil_assaleef
3369.966078492615
47
al_muaras_al_bajela
5421.738415903862
5


### Now using the regular graph as a comparison, run again using the same O-D pairs used with the rehab road graph


In [127]:
rehab_road_tracker_reg = defaultdict(list)

In [128]:
def look_up2(shortest_path, shortest_path_time):
    edge_segements = list(group(shortest_path,2))
    for edge in edge_segements:
        try:           
            if G_adv_w_time[edge[0]][edge[1]][0]['Type'] == 'rehab':
                print('rehab match')
                road_name = G_adv_w_time[edge[0]][edge[1]][0]['name']
                rehab_road_tracker_reg[road_name].append(shortest_path_time)
                break
        except:
            print(G_adv_w_time[edge[0]][edge[1]][0])
            print(edge[0])
            print(edge[1])
            global mismatch_count2
            mismatch_count2 += 1

In [129]:
%time
mismatch_count2 = 0
for origin, destination in rehab_origin_destination_pairs:
    shortest_path = nx.dijkstra_path(G_adv_w_time, origin, destination, weight='time')
    shortest_path_time = OD_df_reg.loc[origin, destination]
    look_up2(shortest_path, shortest_path_time)

Wall time: 0 ns
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match


In [130]:
for key, value in rehab_road_tracker_reg.items():
    print(key)
    print(mean(value))
    print(len(value))

mansuriah_dir_maalm
6831.95963562254
108
bajil_assaleef
4217.022420523155
43
al_muaras_al_bajela
5771.187705099453
5
